In [1]:
import numpy
import math
import pandas
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from sklearn.utils import check_arrays

from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout

Using TensorFlow backend.


In [2]:
numpy.random.seed(7)

dataframe = pandas.read_csv('Paper66_10min.csv', header=None)
dataframe = dataframe.iloc[:, [1,2,4,5,6,7,9,11]]
dataset = dataframe.values
dataset = dataset.astype('float32')

#summary of data
Data_Summary = numpy.zeros((8,3))

for i in range (len(dataset[0, :])):
    Data_Summary[i,0] = numpy.max(dataset[:, i])
    Data_Summary[i,1] = numpy.mean(dataset[:, i])
    Data_Summary[i,2] = numpy.std(dataset[:, i])

print (Data_Summary)

#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)

[[ 1.77149999  0.19888213  0.10472103]
 [ 1.13699996  0.12350018  0.13087003]
 [ 1.14999998  0.23069003  0.27148944]
 [ 0.74599999  0.06392039  0.06839383]
 [ 1.14300001  0.55704874  0.37750435]
 [ 3.29500008  1.46453643  1.03900933]
 [ 0.96950001  0.17980404  0.17077641]
 [ 1.6595      0.27750552  0.35760996]]


In [3]:
def create_dataset(dataset, look_back, step):
    dataX, dataY = [], []
    
    for i in range(len(dataset)-look_back-step):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        b = dataset[i+look_back+step-1, 0]
        dataY.append(b)
    
    return numpy.array(dataX), numpy.array(dataY)

#Nash-Sutcliffe
def NSE(observed, simulated):
    return 1 - numpy.sum((simulated-observed)**2)/numpy.sum((observed-numpy.mean(observed))**2)

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs((y_true - y_pred) / y_true)) * 100

In [4]:
look_back = 48
step = 6

#train, test split
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [5]:
Model_Result = numpy.zeros((8,3))

for i in range (len(dataset[0, :])):
    datasvr = dataset[:, i].reshape((27756,1))
    
    train, test = datasvr[0:train_size,:], datasvr[train_size:len(dataset),:]
    
    TrainX, TrainY = create_dataset(train, look_back, step)
    TestX, TestY = create_dataset(test, look_back, step)
    
    clf = SVR(kernel='rbf', degree=3, 
          gamma=3, coef0=0.0, tol=0.001, 
          C=5, 
          epsilon=0.01, shrinking=True, cache_size=200, verbose=False, max_iter=2500)
    
    clf.fit(TrainX, TrainY)
    svrpredict=clf.predict(TestX)
    
    Model_Result[i,0] = numpy.corrcoef(TestY, svrpredict)[0, 1]
    Model_Result[i,1] = numpy.sqrt(mean_squared_error(TestY, svrpredict))
    Model_Result[i,2] = NSE(TestY, svrpredict)

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=2500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [6]:
print (Model_Result)

[[ 0.77674115  0.06914634 -0.09482193]
 [ 0.60890186  0.07160665  0.28706166]
 [ 0.94557891  0.10411401  0.8656883 ]
 [ 0.84471396  0.03677676  0.70936096]
 [ 0.91644383  0.09064201  0.829515  ]
 [ 0.94992117  0.70417754 -0.07507966]
 [ 0.79124943  0.10057157  0.54964287]
 [ 0.72118692  0.26894559  0.50656966]]
